In [3]:
import pandas as pd
from collections import Counter

import random
import numpy as np
import h5py
import pickle

In [18]:
base_path ='./data/AI_law/sample_seg_train.txt'
def load_data(data_path):
    data =[]
    ids =[]
    max_len =0
    law_labels =[]
    money_label =[]
    with open(data_path,'r',encoding='utf8') as f:
        for line in f:
            line = line.strip()
            lines = line.split('\t')
            ids.append(lines[0])

            sentence = lines[1].split(' ')
            tmp_len = len(sentence)
            if(tmp_len>max_len):
                max_len = tmp_len
            multi_label =[int(i)-1 for i in lines[3].split(',')]
            data.append(sentence)
            law_labels.append(multi_label)
            money_label.append(lines[2])
    return ids , data, money_label,law_labels
ids,data_x,_,data_y = load_data(base_path)

In [66]:
def split_data(datax,labeldata):
    
    indices = np.arange(len(datax))
    np.random.shuffle(indices)
    train_num = int(len(datax)*0.7)
    print(train_num)
    return datax[indices[:train_num],datax[indices[train_num:]],labeldata[indices[:train_num]],labeldata[indices[train_num:]]


SyntaxError: unexpected EOF while parsing (<ipython-input-66-56502f995feb>, line 7)

In [61]:
pklf = open('./word2idx.pkl','rb')
dict_word2index = pickle.load(pklf)
def bulid_dataset(ids,data,labels,dict_word2index,text_maxlen,num_classes):
    datasets=[]
    new_ids =[]
    new_labels =[]
    for i in range(len(labels)):
        new_ids.append('train_'+ids[i])
        multi_label = np.zeros(num_classes)
        for li in labels[i]:
            if(li>num_classes):
                print(li)
                labels[i].remove(li)
        multi_label[labels[i]] =1
        new_labels.append(multi_label)
        new_line =[]
        for word in data[i]:
            if word in dict_word2index:
                new_line.append(dict_word2index[word])
            else:
                new_line.append(0)#UNK
        pad_num = text_maxlen-len(new_line)
        while pad_num>0:
            new_line.append(1)#PAD
            pad_num-=1
        datasets.append(new_line[:text_maxlen])
    return new_ids,np.array(datasets,dtype=np.int64),np.array(new_labels, dtype=np.int64)

ids,data,multilabel = bulid_dataset(ids,data_x,data_y,dict_word2index,text_maxlen=2000,num_classes=452)

2024


2063


3037


2224


In [67]:
train_data_x,valid_data_x,train_data_y,valid_data_y=split_data(data,multilabel)

In [69]:
train_data_y.shape

(7000, 452)

In [76]:
import tensorflow as tf
class Fasttexttf:
    def __init__(self,config):
        #设置超参数
        #self.cofig = config
        self.label_size = config.label_size #e.g.1999
        self.batch_size = config.batch_size
        self.decay_steps, self.decay_rate = config.decay_steps, config.decay_rate
        self.sentence_len=config.sentence_len
        self.vocab_size=config.vocab_size
        self.embed_size=config.embed_size
        self.is_training=config.is_training
        self.learning_rate=config.learning_rate
        self.max_label_per_example=config.max_label_per_example
        self.initializer=tf.random_normal_initializer(stddev=0.1)
        #add 占位符
        self.sentence = tf.placeholder(tf.int32,[None,self.sentence_len],name="sentence")
        self.label = tf.placeholder(tf.int32,[None,self.max_label_per_example],name="Labels")
        self.label_l199 =tf.placeholder(tf.float32,[None,self.label_size])

        # 设置变量
        self.global_step = tf.Variable(0, trainable=False, name="Global_Step")
        self.epoch_step = tf.Variable(0, trainable=False, name="Epoch_Step")
        self.epoch_increment = tf.assign(self.epoch_step, tf.add(self.epoch_step, tf.constant(1)))
        
        self.init_weight()
        self.logits = self.inference()
        self.loss_val = self.loss()
        self.train_op = self.train()


    def init_weight(self):
        self.Embedding = tf.get_variable("Embedding", [self.vocab_size, self.embed_size],initializer=self.initializer)
        self.W = tf.get_variable("W", [self.embed_size, self.label_size],initializer=self.initializer)
        self.b = tf.get_variable("b", [self.label_size])
    def inference(self):
        sentence_embeddings = tf.nn.embedding_lookup(self.Embedding,self.sentence)  # [None,self.sentence_len,self.embed_size]

        # 2.average vectors, to get representation of the sentence
        self.sentence_embeddings = tf.reduce_mean(sentence_embeddings, axis=1)  # [None,self.embed_size]

        # 3.linear classifier layer
        logits = tf.matmul(self.sentence_embeddings, self.W) + self.b #[None, self.label_size]==tf.matmul([None,self.embed_size],[self.embed_size,self.label_size])
        return logits
        
    def loss(self,l2_lambda =0.001):
        labels_multi_hot = self.label_l199  # [batch_size,label_size]
        # sigmoid_cross_entropy_with_logits:Computes sigmoid cross entropy given `logits`.Measures the probability error in discrete classification tasks in which each class is independent and not mutually exclusive.  For instance, one could perform multilabel classification where a picture can contain both an elephant and a dog at the same time.
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_multi_hot,
                                                       logits=self.logits)  # labels:[batch_size,label_size];logits:[batch, label_size]
        loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))  # reduce_sum
        print("loss:", loss)

        # add regularization result in not converge
        self.l2_losses = tf.add_n(
            [tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name]) * l2_lambda
        print("l2_losses:", self.l2_losses)
        loss = loss + self.l2_losses
        return loss
    def train(self):
        learning_rate = tf.train.exponential_decay(self.learning_rate,global_step=self.global_step,decay_rate=self.decay_rate,staircase=True,decay_steps=self.decay_steps)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val, global_step=self.global_step,learning_rate=learning_rate, optimizer="Adam")
        return train_op


In [78]:
import os
class Config:
    label_size = 452
    batch_size =32
    over_sample = True
    sentence_len = 2000
    embed_size = 252
    is_training = True
    learning_rate =0.001
    max_label_per_example = 5
    decay_steps =200
    decay_rate=0.9
    ckpt_dir ='./checkpoints/'
    num_epochs =3
    vocab_size =0
config1=tf.ConfigProto()
config1.gpu_options.allow_growth=True
config2 = Config()
config2.vocab_size= len(dict_word2index)
tf.reset_default_graph()
with tf.Session(config=config1) as sess:
    
    #Instantiate Model
    fast_text=Fasttexttf(config2)
    #Initialize Save
    saver=tf.train.Saver()
    if os.path.exists(config2.ckpt_dir+"checkpoint"):
        print("Restoring Variables from Checkpoint")
        saver.restore(sess,tf.train.latest_checkpoint(config2.ckpt_dir))
    else:
        print('Initializing Variables')
        sess.run(tf.global_variables_initializer())
    curr_epoch=sess.run(fast_text.epoch_step)
    #3.feed data & training
    number_of_training_data=len(train_data_x)
    batch_size=config2.batch_size
    for epoch in range(curr_epoch,config2.num_epochs):#range(start,stop,step_size)
        loss, acc, counter = 0.0, 0.0, 0
        for start, end in zip(range(0, number_of_training_data, batch_size),range(batch_size, number_of_training_data, batch_size)):
            #train_Y_batch=process_labels(trainY[start:end],number=start)
            curr_loss,current_l2_loss,_=sess.run([fast_text.loss_val,fast_text.l2_losses,fast_text.train_op],
                                                 feed_dict={fast_text.sentence:train_data_x[start:end],fast_text.label_l199:train_data_y[start:end]}) #fast_text.labels_l1999:trainY1999[start:end]
            if epoch==0 and counter==0:
                print("trainX[start:end]:",train_data_x[start:end]) #2d-array. each element slength is a 100.
                print("train_Y_batch:",train_data_y[start:end]) #a list,each element is a list.element:may be has 1,2,3,4,5 labels.
                #print("trainY1999[start:end]:",trainY1999[start:end])
            loss,counter=loss+curr_loss,counter+1 #acc+curr_acc,
            if counter %50==0:
                print("Epoch %d\tBatch %d\tTrain Loss:%.3f\tL2 Loss:%.3f" %(epoch,counter,loss/float(counter),current_l2_loss)) #\tTrain Accuracy:%.3f--->,acc/float(counter)           
        print("going to increment epoch counter....")
        sess.run(fast_text.epoch_increment)


loss: Tensor("Mean_1:0", shape=(), dtype=float32)
l2_losses: Tensor("mul:0", shape=(), dtype=float32)


Initializing Variables


trainX[start:end]: [[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]]
train_Y_batch: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Epoch 0	Batch 50	Train Loss:252.597	L2 Loss:25.452


Epoch 0	Batch 100	Train Loss:162.760	L2 Loss:12.109


...

Epoch 2	Batch 50	Train Loss:12.885	L2 Loss:1.366


Epoch 2	Batch 100	Train Loss:12.824	L2 Loss:1.350


Epoch 2	Batch 150	Train Loss:12.785	L2 Loss:1.342


Epoch 2	Batch 200	Train Loss:12.730	L2 Loss:1.339


going to increment epoch counter....
